# Testing Keras CNNs for MNIST dataset

## Importing necessary libraries
[numpy](http://www.numpy.org/), [matplotlib](http://matplotlib.org/) and [Keras](https://keras.io).

In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Import EarlyStopping
from keras.callbacks import EarlyStopping

Set model parameters:

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

Load training data:

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Import saved model

In [5]:
model=keras.models.load_model('models/keras_cnn.h5')

## Define model instead

In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=2)


## Train model with data

In [4]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          #validation_data=(x_test, y_test),
          validation_split=0.2,
          callbacks = [early_stopping_monitor])

model.save('models/keras_cnn.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 83s 1ms/step - loss: 0.2615 - acc: 0.9209 - val_loss: 0.0590 - val_acc: 0.9814
Epoch 2/12
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0856 - acc: 0.9747 - val_loss: 0.0376 - val_acc: 0.9875
Epoch 3/12
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0651 - acc: 0.9803 - val_loss: 0.0342 - val_acc: 0.9887
Epoch 4/12
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0538 - acc: 0.9842 - val_loss: 0.0331 - val_acc: 0.9890
Epoch 5/12
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0471 - acc: 0.9862 - val_loss: 0.0280 - val_acc: 0.9909
Epoch 6/12
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0400 - acc: 0.9878 - val_loss: 0.0317 - val_acc: 0.9894
Epoch 7/12
60000/60000 [==============================] - 81s 1ms/step - loss: 0.0367 - acc: 0.9889 - val_loss: 0.0262 - val_acc

IOError: Unable to create file (unable to open file: name = 'model/keras_cnn.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])